In [2]:
import torch
import numpy as np
import torch.nn as nn
import math
import os
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import json

In [3]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        #does not pass all input words with eachother. Each word goes through independantly
        #and the output are the embeddings of the word. We want this because we do not 
        #want to concacenate the embeddings to the output nodes.
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) 
        #now takes in all embeddings of each word stretched out
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        self.tokens = {}

    def forward(self, inputs):
        #embeds will be flattened matrix
        embeds = self.embeddings(inputs).view((1, -1))
        #rectified relu to learn embeddings
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        #output is the log probablities of all vocabulary
        return log_probs
    def multiply_embedding_weights(self):
        # Multiply the weights of the embedding layer by sqrt(embedding_dim)
        self.embeddings.weight.data = self.embeddings.weight.data * (self.embedding_dim ** 0.5)
    


In [10]:
#file path to credit card csv file
file_path = os.path.join(os.path.expanduser("~"), "Downloads", "mental_health.csv")
orig_dataset = pd.read_csv(file_path) #read csv file as pandas object
orig_dataset = orig_dataset.to_numpy()
print("Dataset size: "+ str(len(orig_dataset)))
dataset = orig_dataset[0:100] #use part of the dataset
print("Train Dataset size: "+ str(len(dataset)))
minFreq = {} #word must appear n times to be added to dictionary
dictionary = {} #relevant words in the dicationary
index = 5
for example in range(len(dataset)):
    for cont_response in range(2):
        if type(dataset[example][cont_response]) == float: #NaN values
            continue
        for word in dataset[example][cont_response].split():
            if word not in minFreq:
                minFreq[word]=1
            else:
                if minFreq[word]==3: #word needs to appear
                    dictionary[word] = index
                    index+=1
                minFreq[word]+=1

print( "Total Dictionary Size: 10,489")
print("Training Dictionary Size: " + str(index))

CONTEXT_SIZE = 3 #look 3 words back to predict current word
EMBEDDING_DIM = 252 #total embeddings for each word
all_ngrams = [] #ngram setup -> [(['through', 'going', "I'm"], 'some')]
for example in range(len(dataset)): 
    for cont_response in range(2): #context than response
        if type(dataset[example][cont_response]) == float: #NaN values
            continue
        cur_Sentence = dataset[example][cont_response].split() #seperate by word
        ngrams = [ #[(['through', 'going', "I'm"], 'some')]
            ([cur_Sentence[i - j - 1] for j in range(CONTEXT_SIZE)],cur_Sentence[i])
            for i in range(CONTEXT_SIZE, len(cur_Sentence))
            ]
        #append the grams to all_ngrams
        for i in ngrams:
            all_ngrams.append(i) 
loss_function = nn.NLLLoss() #loss layer
model = NGramLanguageModeler(index, EMBEDDING_DIM, CONTEXT_SIZE) #intialize Ngram model
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.tokens = dictionary
for epoch in range(10):
    total_loss = 0
    print("Epoch: "+ str(epoch))
    maxFreq = 3 #max number of times a word can be trained
    #dictionary to keep track of times word is trained. Will skip if words have been trained maxFreq times
    maxFreqDict = {}
    for context, target in all_ngrams:
        #if unknown word, just don't train
        if context[0] not in dictionary or context[1] not in dictionary or context[2] not in dictionary:
                continue
        if target not in dictionary:
                continue
        #add context words if not found in dict
        if context[0] not in maxFreqDict:
            maxFreqDict[context[0]] = 1
        if context[1] not in maxFreqDict:
            maxFreqDict[context[1]] = 1
        #if both words have been trained equal to or more than maxFreq times, continue
        #already has been trained enough
        if maxFreqDict[context[0]] >= maxFreq and maxFreqDict[context[1]] >= maxFreq:
            continue
        #update how many times the context words have been trained
        maxFreqDict[context[0]]+=1
        maxFreqDict[context[1]]+=1
            
        #turn each word to an integer and wrapped in tensor so pass as an input to the model
        context_idxs = torch.tensor([dictionary[w] for w in context], dtype=torch.long)

        #zero out gradients cause it accumulates
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        
        
            

        #apply the loss function to the log probabilties with the correct target word
        loss = loss_function(log_probs, torch.tensor([dictionary[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f"Total_Loss: {total_loss}")





Dataset size: 3512
Train Dataset size: 100
Total Dictionary Size: 10,489
Training Dictionary Size: 589
Epoch: 0
Total_Loss: 5906.405691623688
Epoch: 1
Total_Loss: 4417.181990623474
Epoch: 2
Total_Loss: 3579.4913625121117
Epoch: 3
Total_Loss: 3001.517051100731
Epoch: 4
Total_Loss: 2464.736441999674
Epoch: 5
Total_Loss: 1930.116067290306
Epoch: 6
Total_Loss: 1413.5676435604692
Epoch: 7
Total_Loss: 961.7724005840719
Epoch: 8
Total_Loss: 616.9519104510546
Epoch: 9
Total_Loss: 381.9774593450129


In [11]:
model.multiply_embedding_weights()
torch.save(model.state_dict(), "embedding_model")

In [12]:
print(model.embeddings(torch.tensor([0])))

tensor([[ 18.9515, -40.1793,  -3.4174,  35.4660,  -0.3803,  13.8064, -16.4588,
           9.0106,  13.3778,  28.5099,  -2.3675,  17.4134,  20.8600, -11.6594,
          14.0289,  13.5806,  18.0991,  -2.5940,  -2.5052, -16.3638,  21.1693,
           4.1400, -15.8711, -10.0324,  -6.1070,   7.8133,  24.1666, -23.8496,
         -11.6094,  -6.6620,  -6.2208, -31.4289,  -3.7220,  31.1608,  17.8560,
         -21.7332,   4.6913,  20.6389,  -2.3212, -12.1803,   0.1400, -10.8554,
          -6.3194,  -4.5123,  -9.4419,  14.5700,  10.3055,   7.2209,   6.9968,
         -19.7591,  -7.0838,   6.0305, -17.4756, -10.1409,   2.0908,  -1.6202,
         -13.1677, -30.9968, -28.6185,  10.8681,  15.4581, -15.5274,  31.7097,
         -23.2060,   2.2888, -17.3552,  16.2699, -16.3184,  19.5036, -17.8487,
          13.4372,   3.8102, -12.5416,  -0.8427,   8.9910,   0.2782,   7.9117,
           4.7300, -15.8522, -22.5057,   7.8781,  -0.9336,  28.2551,  14.6805,
          -9.8760,  23.1537,   3.0672,   5.6760,  -6

In [13]:
json.dump(model.tokens, open("tokens.txt",'w'))